In [2]:
import numpy as np
import pandas as pd

from scipy.spatial import distance_matrix
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

from scipy.cluster.hierarchy import cophenet
from scipy.spatial.distance import pdist

from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('preprocessed.csv')

n = len(df) #179149

df_slice = df.iloc[:n, :]

products = len(df_slice.code.unique())

In [4]:
df_slice

,order_id,code,quantity,price,pis_cofins,icms,tax_substitution,category,sale_date,source_channel,sale_yearmon,sale_year,sale_month,sale_day,gross_margin,net_margin
0,bcb59c839e78b2601374cbad9239ca7b,e6762ba2ffbca07ab6cee7551caeaad5,1,978.90,90.5483,0.0000,191.8416,4ece547755cba9e7fc14125bc895f31b,2016-06-11,b76eb9b8fc0f17098812da9117d3e500,2016-06,2016,6,11,0.249619,0.353096
1,4e91ee6b95895771dc9ee524e910a902,e6762ba2ffbca07ab6cee7551caeaad5,1,1036.29,95.8568,176.1693,0.0000,4ece547755cba9e7fc14125bc895f31b,2016-06-11,b76eb9b8fc0f17098812da9117d3e500,2016-06,2016,6,11,0.476299,0.213799
2,88eb0ac86af1a521c0831298d22dea8b,e6762ba2ffbca07ab6cee7551caeaad5,1,978.90,90.5483,0.0000,191.8416,4ece547755cba9e7fc14125bc895f31b,2016-06-12,b76eb9b8fc0f17098812da9117d3e500,2016-06,2016,6,12,0.249619,0.353096
3,dee418152a36314b4aee6ce9cf94fcbf,e6762ba2ffbca07ab6cee7551caeaad5,1,978.90,90.5483,176.2020,0.0000,4ece547755cba9e7fc14125bc895f31b,2016-06-13,b76eb9b8fc0f17098812da9117d3e500,2016-06,2016,6,13,0.445596,0.173096
4,1c175bc61b9b659bbf011b2e5e3dcec6,e6762ba2ffbca07ab6cee7551caeaad5,1,976.05,90.2846,0.0000,192.3325,4ece547755cba9e7fc14125bc895f31b,2016-06-13,b76eb9b8fc0f17098812da9117d3e500,2016-06,2016,6,13,0.246925,0.351477
5,a8ad36828898fa3f6efeb5bd19c076f2,e6762ba2ffbca07ab6cee7551caeaad5,1,1089.10,100.7418,185.1470,0.0000,4ece547755cba9e7fc14125bc895f31b,2016-06-13,9d3e0fcbc1f16d80a76026e8f1c26002,2016-06,2016,6,13,0.501693,0.239193
6,9cbfaac6e04ea6ed454b843c94f2c29c,e6762ba2ffbca07ab6cee7551caeaad5,1,949.00,87.7825,170.8200,0.0000,4ece547755cba9e7fc14125bc895f31b,2016-06-14,7261d300057219056592010c7bdaf5ee,2016-06,2016,6,14,0.428128,0.155628
7,66849664f7d51ea5a2959d439d9e079b,e6762ba2ffbca07ab6cee7551caeaad5,1,952.98,88.1507,162.0066,0.0000,4ece547755cba9e7fc14125bc895f31b,2016-09-18,b76eb9b8fc0f17098812da9117d3e500,2016-09,2016,9,18,0.430516,0.168016
8,6bf569a291e777227f9028931e134c92,e6762ba2ffbca07ab6cee7551caeaad5,1,952.98,88.1507,162.0066,0.0000,4ece547755cba9e7fc14125bc895f31b,2016-09-18,b76eb9b8fc0f17098812da9117d3e500,2016-09,2016,9,18,0.430516,0.168016
9,a069e4dc33e94dc5ca166eadda43025c,e6762ba2ffbca07ab6cee7551caeaad5,1,949.00,87.7825,170.8200,0.0000,4ece547755cba9e7fc14125bc895f31b,2016-09-27,98defd6ee70dfb1dea416cecdf391f58,2016-09,2016,9,27,0.428128,0.155628


### Normalizando as variáveis numéricas

In [5]:
normalizable = ['quantity', 'price', 'pis_cofins', 'icms', 'tax_substitution', 'gross_margin', 'net_margin']
df_norm = df_slice.loc[:, normalizable]
df_norm = (df_norm - df_norm.mean()) / df_norm.std()

df_norm = df_norm.drop(['quantity'], axis = 1) # Removing quantity because it distorts the clustering and it's the variable to forecast

### Transformação de variáveis categóricas em dummies

In [6]:
non_normalizable = [c for c in df_slice.columns.tolist() if c not in normalizable]
df_clustering = df_slice.loc[:, non_normalizable]

categorical = ['code', 'category', 'source_channel', 'sale_year', 'sale_month', 'sale_day']
prefix = ['code', 'category', 'source_channel', 'sale_year', 'sale_month', 'sale_day']
df_clustering = pd.get_dummies(df_clustering, columns=categorical, prefix=categorical)

df_clustering = df_clustering.drop(['sale_yearmon', 'sale_date'], axis = 1)

In [7]:
df_clustering = pd.concat([df_clustering, df_norm], axis=1)
df_clustering.head()

,order_id,code_0671c2b9132a3f5215a4212ce0691694,code_09f544ec2a74c89abeec7b0590fc2d11,code_0ad316f6b5cb5e81ebff73ae2490ccfe,code_0bbe09e34a11e8e31cf49d6f8df2992d,code_0dca7ec6ba9b6e8f17f04f713a6be727,code_0f38be2df6854b4374f06cae1bc38482,code_118484c270085e811fbbc81978a269b2,code_13b69fd4bf80b95756e3b138c9169a7f,code_174ef25d9556d516a813e2972f3b8351,...,sale_day_28,sale_day_29,sale_day_30,sale_day_31,price,pis_cofins,icms,tax_substitution,gross_margin,net_margin
0,bcb59c839e78b2601374cbad9239ca7b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3.987707,4.081118,-0.776154,6.185425,-0.205251,0.420172
1,4e91ee6b95895771dc9ee524e910a902,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4.295200,4.386155,4.672400,-0.635450,0.344812,-0.034743
2,88eb0ac86af1a521c0831298d22dea8b,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3.987707,4.081118,-0.776154,6.185425,-0.205251,0.420172
3,dee418152a36314b4aee6ce9cf94fcbf,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3.987707,4.081118,4.673411,-0.635450,0.270308,-0.167671
4,1c175bc61b9b659bbf011b2e5e3dcec6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,3.972437,4.065966,-0.776154,6.202878,-0.211788,0.414886


In [ ]:
Z = linkage(df_clustering.iloc[:, 1:], 'average')

In [ ]:
def fancy_dendrogram(*args, **kwargs):
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if kwargs.get('no_plot', True):
        plt.title('Hierarchical Clustering Dendrogram')
        plt.xlabel('points')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
    return ddata

In [ ]:
fancy_dendrogram(
    Z,
    leaf_rotation=90.,
    leaf_font_size=12.,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=30,  # show only the last p merged clusters
)
plt.show()

### Usar o Coeficiente de Correlação Cofenética para comparar (correlacionar) os pares de distâncias de todas as amostras com aqueles inferidos pelo agrupamento hierarquizado (clustering). Quanto mais próximo de 1, melhor o agrupamento preservou as distâncias originais

In [ ]:
c, coph_dists = cophenet(Z, pdist(df_clustering.iloc[:, 1:]))
c

### Trazer os labels dos agrupamentos de volta para o dataframe

In [ ]:
products = len(df_slice.code.unique())
products

In [ ]:
df_slice['labels'] = fcluster(Z, products, criterion='maxclust')

### Analisando outliers

In [ ]:
df_slice['sale_yearmon'].value_counts()

In [ ]:
outliers_clusters = df_slice['labels'].value_counts()
outliers_clusters = outliers_clusters[outliers_clusters == 1].index.tolist()
df_slice[df_slice['labels'].isin(outliers_clusters)]

### Outliers com preços mais elevados e quantidades maiores

### Separar Mês 05 para Teste de Forecast

In [ ]:
df_slice[df_slice['sale_yearmon'] < '2017-05'][['code', 'quantity', 'sale_date', 'labels']].to_csv('train_clustered.csv', index=False)

In [ ]:
df_slice[df_slice['sale_yearmon'] >= '2017-05'][['code', 'quantity', 'sale_date', 'labels']].to_csv('test_clustered.csv', index=False)